In [ ]:
#@title ###**0. Install software.**
#@markdown Please run this cell by pressing the ▶ _Play_ button on the left.

#@markdown Biopython will be installed.
!pip install biopython > /dev/null
print("Done!")

In [ ]:
from Bio import AlignIO
import pandas as pd
import os
import sys
import argparse
from google.colab import files

#@title ###**1. Input a MSA file and a requirement file.**
#@markdown #####1.1 Upload the MSA and requirement files from "Files" -> "Upload to session storage" on the left.
#@markdown #####1.2 Enter the following fields.
def get_msa(msa_filename=""):
    if msa_filename is None or msa_filename == "":
        print("Select your MSA file!")
        upload_dict = files.upload()
        msa_filename_string = upload_dict[list(upload_dict.keys())[0]]
        with open("tmp.fasta","wb") as out: out.write(msa_filename_string)
        return "tmp.fasta"
    else:
        return msa_filename
def get_requirement(requirement_filename=""):
    if requirement_filename is None or requirement_filename == "":
        print("Select your requirement file!")
        upload_dict = files.upload()
        requirement_filename_string = upload_dict[list(upload_dict.keys())[0]]
        with open("tmp.csv","wb") as out: out.write(requirement_filename_string)
        return "tmp.csv"
    else:
        return requirement_filename
input_fasta = "" #@param {type:"string"}
in_fasta = get_msa(input_fasta)
requirement_file = "" #@param {type:"string"}
in_request = get_requirement(requirement_file)
#@markdown - **<font color="#4682b4">To input MSA file and requirements file on your local PC, run the cell without writing anything in these field and then select the file.</font>**
#@markdown - `input_fasta` Enter the MSA file name. ( e.g. hoge.fasta ) 
#@markdown - `requirements` Enter the requirements file name. ( e.g. hoge.csv ) 
#@markdown - In the csv file, enter `request_name` in the first column and the residue number and the one-letter code of amino acid residue in the second and subsequent columns.　(e.g. request1,43Y,51P,92V,94Y,105F)
df = pd.read_csv(in_request, header=None, index_col=0)
df
#@markdown #####1.3 then, press the ▶ _Play_ button on the left.


In [ ]:
#@title ###**2. Extract sequences from the MSA based on the specified requirements.**

#@markdown ###2.1 Enter the following fields.

request_name="request1" #@param {type:"string"}
#@markdown - `request_name` Enter your request name in the csv file. ( e.g. request1 ) 
output_prefix="result" #@param {type:"string"}
#@markdown - `output_prefix` The result will be saved as <font color="#4682b4">**\<output_prefix\>_\<request_name\>.fasta**</font> in "Files" on the left.  ( e.g. result_request1.fasta ) 

#@markdown ###2.2 Run this cell by pressing the ▶ _Play_ button on the left.
aln = AlignIO.read(open(in_fasta), "fasta")
with open(f"{output_prefix}_{request_name}.fasta", "w") as f:
    for aln_num in range(len(aln)):
        pre_judge=True
        for variant in df.loc[request_name].dropna().to_list():
            if pre_judge and variant[-1] == aln[aln_num].seq[int(variant[:-1])-1]:
                pre_judge=True
            else:
                pre_judge=False
        if pre_judge == True:
            seq_id=aln[aln_num].id
            seq=aln[aln_num].seq
            print(f'>{seq_id}\n{seq}')
            f.write(f'>{seq_id}\n{seq}\n')